In [1]:
# ==============================================================================
# PELATIHAN CNN DETEKSI HAMA: PERBANDINGAN PRE-TRAINED & CUSTOM
# ==============================================================================

print("\n\n")
print("="*70)
print("MEMULAI PELATIHAN CNN DETEKSI HAMA: PERBANDINGAN MODEL")
print("="*70)

# ------------------------------------------------------------------------------
# Langkah 1: Import Library yang dibutuhkan
# ------------------------------------------------------------------------------
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys
import time
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

print(f"TensorFlow Version: {tf.__version__}")




MEMULAI PELATIHAN CNN DETEKSI HAMA: PERBANDINGAN MODEL
TensorFlow Version: 2.16.1


In [2]:
# ------------------------------------------------------------------------------
# Langkah 2: Pre Processing Data (Memuat & Menyiapkan Dataset Hama)
# ------------------------------------------------------------------------------
print("\n--- Langkah 2: Preprocessing Data ---")

# --- Konfigurasi Path Dataset Hama (WAJIB DIUBAH!) ---
DATASET_PEST_DIR = Path('data/pest') # <--- UBAH INI SESUAI LOKASI DATASET ANDA

# --- Parameter Dataset & Preprocessing ---
IMAGE_WIDTH_PEST = 224
IMAGE_HEIGHT_PEST = 224
IMAGE_SIZE_PEST = (IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST)
BATCH_SIZE_PEST = 32
# NUM_CLASSES_PEST akan diambil dari direktori nanti

# --- Memuat Dataset ---
print(f"Mencari dataset hama di: {DATASET_PEST_DIR}")
TRAIN_DIR_PEST = DATASET_PEST_DIR / 'train'
VAL_DIR_PEST = DATASET_PEST_DIR / 'test' # Menggunakan folder 'test' sebagai validasi

if not TRAIN_DIR_PEST.is_dir() or not VAL_DIR_PEST.is_dir():
    print(f"Error: Folder 'train' atau 'test' (untuk validasi) tidak ditemukan di {DATASET_PEST_DIR}")
    sys.exit(1)

try:
    print("Memuat data training...")
    train_ds_pest = tf.keras.utils.image_dataset_from_directory(
        TRAIN_DIR_PEST, seed=123, image_size=IMAGE_SIZE_PEST,
        batch_size=BATCH_SIZE_PEST, label_mode='categorical'
    )
    print("Memuat data validasi...")
    val_ds_pest = tf.keras.utils.image_dataset_from_directory(
        VAL_DIR_PEST, seed=123, image_size=IMAGE_SIZE_PEST,
        batch_size=BATCH_SIZE_PEST, label_mode='categorical', shuffle=False
    )
except Exception as e:
    print(f"Error saat memuat dataset hama: {e}")
    sys.exit(1)

CLASS_NAMES_PEST = train_ds_pest.class_names
NUM_CLASSES_PEST = len(CLASS_NAMES_PEST) # Ambil jumlah kelas dari dataset
print(f"Kelas hama ditemukan ({NUM_CLASSES_PEST}): {CLASS_NAMES_PEST}")

# --- Optimasi & Augmentasi ---
AUTOTUNE = tf.data.AUTOTUNE
train_ds_pest = train_ds_pest.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds_pest = val_ds_pest.cache().prefetch(buffer_size=AUTOTUNE)

# Layer augmentasi data - pastikan RandomRotation tidak ada value_range
data_augmentation_pest = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1), # Sudah benar, tidak ada value_range
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.1),
    ], name="data_augmentation"
)
print("Dataset dioptimalkan dan layer augmentasi dibuat.")


--- Langkah 2: Preprocessing Data ---
Mencari dataset hama di: data\pest
Memuat data training...
Found 2100 files belonging to 7 classes.
Memuat data validasi...
Found 350 files belonging to 7 classes.
Kelas hama ditemukan (7): ['kutu_daun', 'kutu_kebul', 'lalat_buah', 'thrips', 'tungau', 'ulat_grayak', 'ulat_penggerek_buah']
Dataset dioptimalkan dan layer augmentasi dibuat.


In [3]:
# ------------------------------------------------------------------------------
# Langkah 3: Definisikan Struktur Model (Sesuai dengan yang akan direkonstruksi di VPS)
# ------------------------------------------------------------------------------
print("\n--- Langkah 3: Definisi Struktur Model CNN Hama ---")

NAMA_MODEL = 'MobileNetV2'  # Ganti ke 'ResNet50V2', 'EfficientNetB0', 'VGG16', atau 'CustomCNN'
# NAMA_MODEL = 'ResNet50V2'

print(f"Membangun model menggunakan: {NAMA_MODEL}")
inputs_pest = tf.keras.Input(shape=(IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST, 3), name="input_gambar_hama", dtype=tf.float32)
x = data_augmentation_pest(inputs_pest) # Output augmentasi masih 0-255

base_model_pest = None
preprocess_input_specific = None
train_base_model = False # Set True jika Anda ingin melakukan fine-tuning pada base model

if NAMA_MODEL == 'MobileNetV2':
    # MobileNetV2 mengharapkan input ternormalisasi ke [-1, 1]
    x_processed = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    base_model_pest = tf.keras.applications.MobileNetV2(input_tensor=x_processed, # Langsung sambungkan
                                                        input_shape=(IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST, 3),
                                                        include_top=False, weights='imagenet')
elif NAMA_MODEL == 'ResNet50V2':
    # ResNet50V2 menggunakan preprocessing gaya Caffe
    x_processed = tf.keras.applications.resnet_v2.preprocess_input(x)
    base_model_pest = tf.keras.applications.ResNet50V2(input_tensor=x_processed, # Langsung sambungkan
                                                       input_shape=(IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST, 3),
                                                       include_top=False, weights='imagenet')
elif NAMA_MODEL == 'EfficientNetB0':
    x_processed = tf.keras.applications.efficientnet.preprocess_input(x)
    base_model_pest = tf.keras.applications.EfficientNetB0(input_tensor=x_processed, # Langsung sambungkan
                                                           input_shape=(IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST, 3),
                                                           include_top=False, weights='imagenet')
elif NAMA_MODEL == 'VGG16':
    x_processed = tf.keras.applications.vgg16.preprocess_input(x)
    base_model_pest = tf.keras.applications.VGG16(input_tensor=x_processed, # Langsung sambungkan
                                                  input_shape=(IMAGE_WIDTH_PEST, IMAGE_HEIGHT_PEST, 3),
                                                  include_top=False, weights='imagenet')
elif NAMA_MODEL == 'CustomCNN':
    # Jika custom, pastikan Anda menangani rescaling/normalisasi jika perlu
    x_processed = layers.Rescaling(1./255, name="rescaling")(x) # Contoh rescaling untuk custom CNN
    # Definisikan layer custom CNN Anda di sini menggantikan base_model_pest
    # Contoh:
    c = layers.Conv2D(32, (3, 3), activation='relu', name="conv1")(x_processed)
    c = layers.MaxPooling2D((2, 2), name="pool1")(c)
    c = layers.Conv2D(64, (3, 3), activation='relu', name="conv2")(c)
    c = layers.MaxPooling2D((2, 2), name="pool2")(c)
    c = layers.Conv2D(128, (3, 3), activation='relu', name="conv3")(c)
    output_base_model_equivalent = layers.MaxPooling2D((2, 2), name="pool3")(c)
    print("Arsitektur Custom CNN sederhana dibuat.")
else:
    print(f"Error: Nama model '{NAMA_MODEL}' tidak dikenali.")
    sys.exit(1)

# Handling output dari base model atau custom model
if NAMA_MODEL != 'CustomCNN' and base_model_pest:
    base_model_pest.trainable = train_base_model # Bekukan atau unfreeze base model
    output_base_model_equivalent = base_model_pest.output
    print(f"Base model {NAMA_MODEL} dimuat, trainable={train_base_model}.")
elif NAMA_MODEL == 'CustomCNN':
    pass # output_base_model_equivalent sudah di-assign di atas
else: # Jika base_model_pest gagal diinisialisasi karena NAMA_MODEL salah
    print("Base model tidak terdefinisi dengan benar.")
    sys.exit(1)


# Top Layers (Classifier) - PASTIKAN INI SAMA DENGAN YANG AKAN ANDA BUAT DI VPS
x_top = layers.GlobalAveragePooling2D(name="global_pooling")(output_base_model_equivalent)
x_top = layers.Dropout(0.3, name="dropout_top")(x_top) # Sesuaikan dropout rate

outputs_pest = layers.Dense(NUM_CLASSES_PEST, activation='softmax', name="output_prediksi")(x_top)

model_pest = tf.keras.Model(inputs=inputs_pest, outputs=outputs_pest, name=f"PestDetector_{NAMA_MODEL}")

print(f"Model final ({NAMA_MODEL}) selesai dibangun.")
model_pest.summary()


--- Langkah 3: Definisi Struktur Model CNN Hama ---
Membangun model menggunakan: MobileNetV2
Base model MobileNetV2 dimuat, trainable=False.
Model final (MobileNetV2) selesai dibangun.


Model: "PestDetector_MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_gambar_hama   │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ data_augmentation   │ (None, 224, 224,  │          0 │ input_gambar_ham… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ true_divide         │ (None, 224, 224,  │          0 │ data_augmentatio… │
│ (TrueDivide)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract (Subtract) │ (None, 224, 224,  │          0 │ true_divide[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ subtract[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0

 Total params: 2,266,951 (8.65 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
# ------------------------------------------------------------------------------
# Langkah 4: Train Model
# ------------------------------------------------------------------------------
print(f"\n--- Langkah 4: Train Model ({NAMA_MODEL}) ---")

EPOCHS_PEST = 50 # Jumlah epoch bisa disesuaikan (misal, 50 untuk contoh)
LEARNING_RATE_PEST = 0.001

model_pest.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_PEST),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(f"Model ({NAMA_MODEL}) dikompilasi.")

print(f"Memulai training {NAMA_MODEL} untuk max {EPOCHS_PEST} epochs...")
start_train_time = time.time()

# Definisikan path untuk menyimpan model .keras terbaik dan bobot .weights.h5 terbaik
BEST_MODEL_KERAS_PATH = f'hama_cabai_{NAMA_MODEL}_best_e{EPOCHS_PEST}.keras'
BEST_MODEL_WEIGHTS_PATH = f'hama_cabai_{NAMA_MODEL}_best_e{EPOCHS_PEST}.weights.h5'

callbacks_list = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_MODEL_KERAS_PATH, # Simpan seluruh model terbaik
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    # Tambahan: Callback untuk menyimpan bobot terbaik secara terpisah
    tf.keras.callbacks.ModelCheckpoint(
        filepath=BEST_MODEL_WEIGHTS_PATH, # Simpan HANYA BOBOT terbaik
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)
]

history_pest = model_pest.fit(
    train_ds_pest,
    validation_data=val_ds_pest,
    epochs=EPOCHS_PEST,
    callbacks=callbacks_list,
    verbose=1
)
end_train_time = time.time()
training_duration = end_train_time - start_train_time
actual_epochs_run = len(history_pest.history['loss']) # Dapatkan jumlah epoch aktual yang dijalankan
print(f"Training {NAMA_MODEL} selesai dalam {training_duration:.2f} detik ({actual_epochs_run} epochs dijalankan).")

# --- Simpan Model Final dan Bobot Final (setelah semua epoch atau setelah early stopping) ---
# Model terbaik sudah disimpan oleh ModelCheckpoint.
# Jika Anda ingin menyimpan model pada state terakhir training (bukan yang terbaik berdasarkan val_accuracy):
FINAL_MODEL_KERAS_PATH = f'hama_cabai_{NAMA_MODEL}_final_e{actual_epochs_run}.keras'
model_pest.save(FINAL_MODEL_KERAS_PATH)
print(f"Model Keras final (epoch terakhir) disimpan di: {FINAL_MODEL_KERAS_PATH}")

FINAL_MODEL_WEIGHTS_PATH = f'hama_cabai_{NAMA_MODEL}_final_e{actual_epochs_run}.weights.h5'
model_pest.save_weights(FINAL_MODEL_WEIGHTS_PATH)
print(f"Bobot model final (epoch terakhir) disimpan di: {FINAL_MODEL_WEIGHTS_PATH}")

print(f"\nFile bobot TERBAIK yang sangat disarankan untuk diupload ke VPS adalah: {BEST_MODEL_WEIGHTS_PATH}")
print(f"(Anda juga memiliki bobot dari epoch terakhir di: {FINAL_MODEL_WEIGHTS_PATH})")


--- Langkah 4: Train Model (MobileNetV2) ---
Model (MobileNetV2) dikompilasi.
Memulai training MobileNetV2 untuk max 50 epochs...
Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step - accuracy: 0.3468 - loss: 1.7572
Epoch 1: val_accuracy improved from -inf to 0.81143, saving model to hama_cabai_MobileNetV2_best_e50.keras

Epoch 1: val_accuracy improved from -inf to 0.81143, saving model to hama_cabai_MobileNetV2_best_e50.weights.h5
66/66 ━━━━━━━━━━━━━━━━━━━━ 40s 495ms/step - accuracy: 0.3491 - loss: 1.7510 - val_accuracy: 0.8114 - val_loss: 0.6441 - learning_rate: 0.0010
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.7602 - loss: 0.7133
Epoch 2: val_accuracy improved from 0.81143 to 0.87143, saving model to hama_cabai_MobileNetV2_best_e50.keras

Epoch 2: val_accuracy improved from 0.81143 to 0.87143, saving model to hama_cabai_MobileNetV2_best_e50.weights.h5
66/66 ━━━━━━━━━━━━━━━━━━━━ 30s 456ms/step - accuracy: 0.7604 - loss: 0.7125 - val_accuracy: 0.8714 - val_loss

In [5]:
# ------------------------------------------------------------------------------
# Langkah 5: Evaluasi Model (LENGKAP)
# ------------------------------------------------------------------------------
print(f"\n--- Langkah 5: Evaluasi Model Lengkap ({NAMA_MODEL}) ---")
# --- Memuat model terbaik untuk evaluasi ---
# Karena EarlyStopping punya restore_best_weights=True, 'model_pest' sudah merupakan model terbaik
# Namun, jika Anda ingin lebih yakin atau menggunakan file yang disimpan ModelCheckpoint:
print(f"Memuat model terbaik dari {BEST_MODEL_KERAS_PATH} untuk evaluasi akhir...")
try:
    # Saat memuat model .keras yang disimpan di lingkungan yang sama, custom_objects biasanya tidak diperlukan.
    best_model_to_evaluate = tf.keras.models.load_model(BEST_MODEL_KERAS_PATH)
    print("Model terbaik berhasil dimuat.")
except Exception as e_load_best:
    print(f"Gagal memuat model terbaik dari {BEST_MODEL_KERAS_PATH}, menggunakan model saat ini dari training. Error: {e_load_best}")
    best_model_to_evaluate = model_pest # Fallback ke model terakhir jika load gagal

loss_pest, accuracy_pest = best_model_to_evaluate.evaluate(val_ds_pest, verbose=0)
# ... (Sisa kode evaluasi Anda dari Langkah 5 tetap sama, gunakan best_model_to_evaluate untuk predict) ...
print(f'\nHasil Evaluasi Umum pada Set Validasi/Test (Model Terbaik):')
print(f'  Akurasi Keseluruhan: {accuracy_pest*100:.2f}%')
print(f'  Loss Keseluruhan   : {loss_pest:.4f}')

print("\nMenghitung metrik evaluasi detail (Confusion Matrix & Classification Report) menggunakan model terbaik...")
y_true_all = []
y_pred_all = []
has_data_for_eval = False
for images_batch, labels_batch in val_ds_pest:
    has_data_for_eval = True
    true_indices_batch = np.argmax(labels_batch.numpy(), axis=1)
    y_true_all.extend(true_indices_batch)
    preds_batch = best_model_to_evaluate.predict(images_batch, verbose=0) # Gunakan best_model_to_evaluate
    pred_indices_batch = np.argmax(preds_batch, axis=1)
    y_pred_all.extend(pred_indices_batch)

if has_data_for_eval and len(y_true_all) > 0:
    print(f"Memproses {len(y_true_all)} sampel dari validation set.")
    try:
        cm = confusion_matrix(y_true_all, y_pred_all, labels=range(NUM_CLASSES_PEST))
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=CLASS_NAMES_PEST, yticklabels=CLASS_NAMES_PEST)
        plt.xlabel("Prediksi Model"); plt.ylabel("Label Sebenarnya")
        plt.title(f"Confusion Matrix - Model {NAMA_MODEL} (Best)")
        plt.tight_layout()
        cm_filename = f"confusion_matrix_{NAMA_MODEL}_best_e{actual_epochs_run}.png"
        plt.savefig(cm_filename); print(f"Confusion Matrix disimpan ke: {cm_filename}")
        plt.close()
    except Exception as e_cm: print(f"Gagal membuat Confusion Matrix: {e_cm}")

    try:
        report = classification_report(y_true_all, y_pred_all,
                                       target_names=CLASS_NAMES_PEST,
                                       labels=range(NUM_CLASSES_PEST),
                                       zero_division=0)
        print("\n=== CLASSIFICATION REPORT (Model Terbaik) ===")
        print(report)
        report_filename = f"classification_report_{NAMA_MODEL}_best_e{actual_epochs_run}.txt"
        with open(report_filename, "w") as f:
            f.write(f"Classification Report for Model: {NAMA_MODEL} (Best)\n"); f.write(f"Epochs Run: {actual_epochs_run}\n")
            f.write(f"Overall Accuracy: {accuracy_pest*100:.2f}%\n"); f.write(f"Overall Loss: {loss_pest:.4f}\n\n"); f.write(report)
        print(f"Classification Report disimpan ke: {report_filename}")
    except Exception as e_report: print(f"Gagal membuat Classification Report: {e_report}")
else: print("Tidak ada data untuk evaluasi detail.")

# --- Plot Hasil Pelatihan ---
if 'accuracy' in history_pest.history and 'val_accuracy' in history_pest.history:
    try:
        # ... (kode plotting sama seperti sebelumnya, pastikan menggunakan actual_epochs_run untuk range) ...
        acc = history_pest.history['accuracy']
        val_acc = history_pest.history['val_accuracy']
        loss = history_pest.history['loss']
        val_loss = history_pest.history['val_loss']
        epochs_range_pest = range(actual_epochs_run)

        plt.figure(figsize=(12, 5))
        plt.suptitle(f'Training History for {NAMA_MODEL}', fontsize=16)
        plt.subplot(1, 2, 1)
        plt.plot(epochs_range_pest, acc, label='Training Accuracy')
        plt.plot(epochs_range_pest, val_acc, label='Validation Accuracy')
        plt.legend(loc='lower right'); plt.title('Accuracy')
        plt.xlabel(f'Epoch (Ran: {actual_epochs_run})'); plt.ylabel('Accuracy')
        plt.subplot(1, 2, 2)
        plt.plot(epochs_range_pest, loss, label='Training Loss')
        plt.plot(epochs_range_pest, val_loss, label='Validation Loss')
        plt.legend(loc='upper right'); plt.title('Loss')
        plt.xlabel(f'Epoch (Ran: {actual_epochs_run})'); plt.ylabel('Loss')

        plot_filename = f'training_history_{NAMA_MODEL}_e{actual_epochs_run}.png'
        plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to make space for suptitle
        plt.savefig(plot_filename)
        print(f"Plot histori training disimpan ke: {plot_filename}")
        plt.close() # Tutup plot agar tidak ditampilkan jika ini dijalankan sebagai skrip
    except Exception as e_plot: print(f"Gagal membuat plot histori training: {e_plot}")
else: print("Tidak dapat membuat plot histori, data akurasi tidak ditemukan.")



--- Langkah 5: Evaluasi Model Lengkap (MobileNetV2) ---
Memuat model terbaik dari hama_cabai_MobileNetV2_best_e50.keras untuk evaluasi akhir...
Model terbaik berhasil dimuat.

Hasil Evaluasi Umum pada Set Validasi/Test (Model Terbaik):
  Akurasi Keseluruhan: 98.29%
  Loss Keseluruhan   : 0.1614

Menghitung metrik evaluasi detail (Confusion Matrix & Classification Report) menggunakan model terbaik...
Memproses 350 sampel dari validation set.
Confusion Matrix disimpan ke: confusion_matrix_MobileNetV2_best_e38.png

=== CLASSIFICATION REPORT (Model Terbaik) ===
                     precision    recall  f1-score   support

          kutu_daun       1.00      1.00      1.00        50
         kutu_kebul       1.00      1.00      1.00        50
         lalat_buah       0.96      1.00      0.98        50
             thrips       1.00      1.00      1.00        50
             tungau       1.00      0.92      0.96        50
        ulat_grayak       0.96      1.00      0.98        50
ulat_pe

In [7]:
# ------------------------------------------------------------------------------
# Langkah 6: Testing Model (Contoh Inference)
# ------------------------------------------------------------------------------
# Pastikan MODEL_SAVE_PATH_PEST menunjuk ke model terbaik jika itu yang ingin diuji.
# Di sini kita akan menggunakan BEST_MODEL_KERAS_PATH untuk konsistensi dengan evaluasi.
print(f"\n--- Langkah 6: Testing Model {NAMA_MODEL} ---")
# ... (Sisa kode Langkah 6 Anda untuk memilih gambar uji acak tetap sama) ...
import cv2

TEST_IMAGE_PEST_PATH_EXAMPLE = None
try:
    list_kelas_test = [d for d in VAL_DIR_PEST.iterdir() if d.is_dir()]
    if list_kelas_test:
        kelas_terpilih = np.random.choice(list_kelas_test)
        list_gambar_test = list(kelas_terpilih.glob('*.jpg')) + list(kelas_terpilih.glob('*.png')) + list(kelas_terpilih.glob('*.jpeg'))
        if list_gambar_test:
             TEST_IMAGE_PEST_PATH_EXAMPLE = str(np.random.choice(list_gambar_test))
             print(f"Gambar uji dipilih secara acak: {TEST_IMAGE_PEST_PATH_EXAMPLE}")
        else: print(f"Tidak ada gambar di folder kelas {kelas_terpilih.name}")
    else: print("Tidak ada folder kelas di direktori validasi/test.")
except Exception as e_find: print(f"Gagal memilih gambar uji acak: {e_find}")


if Path(BEST_MODEL_KERAS_PATH).is_file() and TEST_IMAGE_PEST_PATH_EXAMPLE and Path(TEST_IMAGE_PEST_PATH_EXAMPLE).is_file():
    print(f"Memuat model {NAMA_MODEL} dari: {BEST_MODEL_KERAS_PATH}")
    try:
        # Tidak perlu custom_objects saat memuat di lingkungan yang sama dengan training
        loaded_model_for_testing = tf.keras.models.load_model(BEST_MODEL_KERAS_PATH)
        print(f"Model {NAMA_MODEL} (terbaik) berhasil dimuat untuk testing.")
        
        print(f"Menjalankan inference pada gambar: {TEST_IMAGE_PEST_PATH_EXAMPLE}")
        img_pest_test = tf.keras.utils.load_img(TEST_IMAGE_PEST_PATH_EXAMPLE, target_size=IMAGE_SIZE_PEST)
        img_array_pest = tf.keras.utils.img_to_array(img_pest_test) # dtype=float32 by default
        img_batch_pest = tf.expand_dims(img_array_pest, 0) # Buat batch
        
        start_pred_time = time.time()
        predictions_pest = loaded_model_for_testing.predict(img_batch_pest)
        end_pred_time = time.time()
        
        score_pest = tf.nn.softmax(predictions_pest[0])
        predicted_class_index = np.argmax(score_pest)
        predicted_class_name = CLASS_NAMES_PEST[predicted_class_index]
        confidence_score = 100 * np.max(score_pest)
        
        print(f"\nHasil Prediksi {NAMA_MODEL} (Inferensi):")
        print(f"  Gambar       : {Path(TEST_IMAGE_PEST_PATH_EXAMPLE).name}")
        print(f"  Kelas Prediksi: {predicted_class_name}")
        print(f"  Kepercayaan  : {confidence_score:.2f}%")
        print(f"  Waktu Inferensi: {end_pred_time - start_pred_time:.4f} detik")
        
        try:
            img_display = cv2.imread(TEST_IMAGE_PEST_PATH_EXAMPLE)
            if img_display is not None:
                 cv2.putText(img_display, f"Prediksi ({NAMA_MODEL}): {predicted_class_name} ({confidence_score:.2f}%)",(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                 output_filename_pest = f"test_result_{NAMA_MODEL}_{Path(TEST_IMAGE_PEST_PATH_EXAMPLE).stem}.jpg"
                 cv2.imwrite(output_filename_pest, img_display)
                 print(f"  Gambar hasil test disimpan ke: {output_filename_pest}")
            else: print("  Gagal memuat gambar untuk ditampilkan/disimpan.")
        except Exception as e_cv: print(f"  Error saat menampilkan/menyimpan gambar test: {e_cv}")

    except Exception as e:
        print(f"Error saat testing/inference {NAMA_MODEL}: {e}")
        import traceback; traceback.print_exc()
elif not TEST_IMAGE_PEST_PATH_EXAMPLE: print("Testing dilewati, tidak ada gambar uji.")
else: print(f"Model terbaik '{BEST_MODEL_KERAS_PATH}' tidak ditemukan. Testing dilewati.")


print(f"\n--- SEMUA PROSES UNTUK MODEL {NAMA_MODEL} SELESAI ---")


--- Langkah 6: Testing Model MobileNetV2 ---
Gambar uji dipilih secara acak: data\pest\test\ulat_penggerek_buah\jpg_40.jpg
Memuat model MobileNetV2 dari: hama_cabai_MobileNetV2_best_e50.keras
Model MobileNetV2 (terbaik) berhasil dimuat untuk testing.
Menjalankan inference pada gambar: data\pest\test\ulat_penggerek_buah\jpg_40.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

Hasil Prediksi MobileNetV2 (Inferensi):
  Gambar       : jpg_40.jpg
  Kelas Prediksi: ulat_penggerek_buah
  Kepercayaan  : 31.09%
  Waktu Inferensi: 1.5472 detik
  Gambar hasil test disimpan ke: test_result_MobileNetV2_jpg_40.jpg

--- SEMUA PROSES UNTUK MODEL MobileNetV2 SELESAI ---
